That notebook is going to be used for the capstone project of IBM data science certification course

In [1]:
import pandas as pd
import numpy as np

In [2]:
print('Hello Capstone Project Course')

Hello Capstone Project Course


<h2>Setup </h2>
<p>BeautifulSoup library is used for web scraping. It is imported as soup<p>
To access the wikipedia web page, urlopen is imported from urllib 

In [14]:
from bs4 import BeautifulSoup as soup
from urllib.request import Request,urlopen

In [27]:
wiki_url = 'http://en.turkcewiki.org/wiki/List_of_postal_codes_of_Canada:_M'

#user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'
hdr = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'

      }
req = Request(url=wiki_url, headers=hdr)
wiki_data = urlopen(req).read()

In [30]:
#wiki_html = urlopen(wiki_data).read()
#wiki_data.close()

AttributeError: 'Request' object has no attribute 'close'

<h2>Web Scraping</h2><p>Following cells perform the web scraping. From Wikipedia file, the table element scraped by its class name<p>

In [31]:
page_soup=soup(wiki_data, 'html.parser')

In [32]:
postCode_table=page_soup.findAll('table', {'class':"wikitable sortable"})

In [33]:
postCode_table = postCode_table[0]

Name of the columns are extracted from html table

In [34]:
cols = postCode_table.findAll('th', {})
print(cols)

[<th>Postcode</th>, <th>Borough</th>, <th>Neighbourhood
</th>]


<p>Tags of items in table are removed by using following for loop<p>

In [35]:
cols_name = []
for col in cols:
    cols_name.append(col.text)
print(cols_name)

['Postcode', 'Borough', 'Neighbourhood\n']


In [36]:
cols_name[2] = cols_name[2].replace('\n','')

In [37]:
type(cols_name)

list

Items under each columns are extracted from the html table

In [38]:
all_rows = postCode_table.findAll('tr', {})

In [39]:
all_rows=all_rows[1:]

In [40]:
first_row = all_rows[0]

In [41]:
first_row_data = first_row.findAll('td',{})

In [42]:
first_row_data

[<td>M1A</td>, <td>Not assigned</td>, <td>Not assigned
 </td>]

In [43]:
rows_data = []
for data in first_row_data:
    rows_data.append(data.text)
print(rows_data)

['M1A', 'Not assigned', 'Not assigned\n']


In [44]:
rows_data[2] = rows_data[2].replace('\n','')

In [45]:
type(rows_data)

list

Function to handle each and every single rows of columns from the table

In [46]:
def row_gen(row):
    row_data = row.findAll('td',{})
    r_data = []
    for d in row_data:
        r_data.append(d.text)
    r_data[2] = r_data[2].replace('\n','')
    return r_data

Gathering every rows generated

In [47]:
rows_tobe = []
for i in all_rows:
    rows_tobe.append(row_gen(i))

A datafrane created by all of rows and columns 

In [48]:
df_rows = pd.DataFrame(rows_tobe,columns = cols_name)

In [49]:
df_rows.head(5) #to Check the general appearence of table from head 

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [50]:
df_rows.tail(5)  #to Check the general appearence of table from head 

,Postcode,Borough,Neighbourhood
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor
286,M9Z,Not assigned,Not assigned


Not assigned boroughs are removed from dataframe

In [51]:
df_rows = df_rows[df_rows['Borough'] != 'Not assigned']

Not assigned Neighbourhoods are assigned as their borough's name

In [52]:
for i in df_rows.index:
    if df_rows.loc[i,'Neighbourhood'] == 'Not assigned':
        df_rows.loc[i,'Neighbourhood'] = df_rows.loc[i,'Borough']
        

In [53]:
df_rows[df_rows['Neighbourhood']==df_rows['Borough']]

,Postcode,Borough,Neighbourhood
7,M7A,Queen's Park,Queen's Park
9,M9A,Queen's Park,Queen's Park


Items are grouped by their postcodes and then neighbourhoods with same postal code are combined into a single row

In [54]:
df_rows= df_rows.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

In [55]:
df_rows.head(5)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


<h2>Accessing the location data</h2>
<p>Because of unreliable package, continuing with csv file is decided after a couple of tries. <p>
<p>Wget which we commonly used in labs installed, in order to download csv provided.<p>

In [56]:
conda install -c anaconda wget

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [57]:
!wget -O coordinates_neighbourhood.csv http://cocl.us/Geospatial_data

--2019-11-19 19:38:43--  http://cocl.us/Geospatial_data
Resolving cocl.us... 159.8.72.228
Connecting to cocl.us|159.8.72.228|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2019-11-19 19:38:43--  https://cocl.us/Geospatial_data
Connecting to cocl.us|159.8.72.228|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-11-19 19:38:46--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com... 185.235.236.197
Connecting to ibm.box.com|185.235.236.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-11-19 19:38:47--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:44

A dataframe which consists location data of each item is created

In [58]:
df_coord = pd.read_csv("coordinates_neighbourhood.csv")
df_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [59]:
df_coord.shape

(103, 3)

Coordinates are concatenated to main dataframe to obtain requested dataframe 

In [60]:
df_rows = pd.concat([df_rows,df_coord['Latitude']],axis = 1)

In [61]:
df_rows = pd.concat([df_rows,df_coord['Longitude']],axis = 1)

In [62]:
df_rows.head(12)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


<h2>Shape of the main dataframe is as follows </h2>

In [73]:
df_toronto = df_rows[df_rows['Borough'].str.contains(pat='Toronto',regex=False)]

In [74]:
df_toronto

,Postcode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
49,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


In [ ]:
df_rows.shape

In [ ]:
df_rows.to_csv('\neighbourhood_rows.csv',index = None)

In [77]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: \ 
  - anaconda::ca-certificates-2019.8.28-0, anaconda::openssl-1.1.1d-h1de35cc_2
  - anaconda::ca-certificates-2019.8.28-0, defaults::openssl-1.1.1d-h1de35cc_2
  - anaconda::openssl-1.1.1d-h1de35cc_2, defaults::ca-certificates-2019.8.28-0
  - defaults::ca-certificates-2019.8.28-0, defaults::openssl-1.1.1d-h1de35ccdone

## Package Plan ##

  environment location: /Users/oguzonuk/opt/anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-3.2.0               |           py37_0         773 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    certifi-2019.9.11          |           py37_0         147 KB  conda-forge
    conda-4.7.12               |           py37_0         3.0 MB  conda-f

In [80]:
# create map of Toronto using latitude and longitude values
latitude=43.6532
longitude=-79.3832
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [85]:
CLIENT_ID = '3LBBKMPI2EYBRBFMBBODMS3402O2ODMVGRSXY0CGWT3P2MIM' # your Foursquare ID
CLIENT_SECRET = '2LRKRPSZLXEB31FBQQIXDUTZ3NEJ2YL15IO1325V3FT534T4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3LBBKMPI2EYBRBFMBBODMS3402O2ODMVGRSXY0CGWT3P2MIM
CLIENT_SECRET:2LRKRPSZLXEB31FBQQIXDUTZ3NEJ2YL15IO1325V3FT534T4


In [88]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
radius = 500
LIMIT = 100

In [81]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [89]:
# type your answer here

toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )



The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The Junction Sout

In [92]:
toronto_venues.head(5)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [93]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 230 uniques categories.


In [94]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [95]:
toronto_onehot.shape

(1703, 230)

In [96]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [97]:
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,...,0.000000,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.000000,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Central Bay Street,0.012500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012500,...,0.000000,0.000000,0.00000,0.00,0.012500,0.000000,0.000000,0.012500,0.000000,0.000000
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.041667,0.000000,0.052083,0.010417,0.000000,0.000000
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Church and Wellesley,0.011494,0.011494,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011494,...,0.011494,0.000000,0.00000,0.00,0.000000,0.011494,0.011494,0.000000,0.011494,0.011494


In [98]:
toronto_grouped.shape

(38, 230)

In [99]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [101]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Steakhouse,Breakfast Spot,Asian Restaurant,Hotel,Restaurant,Cosmetics Shop,Bakery
1,Berczy Park,Coffee Shop,Bakery,Steakhouse,Cocktail Bar,Cheese Shop,Café,Seafood Restaurant,Beer Bar,Farmers Market,Breakfast Spot
2,"Brockton, Exhibition Place, Parkdale Village",Performing Arts Venue,Café,Coffee Shop,Bakery,Breakfast Spot,Gym,Furniture / Home Store,Convenience Store,Burrito Place,Stadium
3,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Spa,Gym / Fitness Center,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Light Rail Station,Comic Shop,Park
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Airport Terminal,Plane,Bar,Coffee Shop,Sculpture Garden,Boutique,Boat or Ferry,Harbor / Marina


In [102]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [107]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood


ValueError: cannot insert Cluster Labels, already exists

In [106]:
df_toronto.rename(columns={'Neighbourhood':'Neighborhood'},inplace=True)

/Users/oguzonuk/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


In [108]:
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Health Food Store,Trail,Pub,Wings Joint,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Bubble Tea Shop,Indian Restaurant,Spa,Juice Bar,Bookstore
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Sandwich Place,Brewery,Fish & Chips Shop,Food & Drink Shop,Sushi Restaurant,Ice Cream Shop,Fast Food Restaurant,Pub,Movie Theater,Italian Restaurant
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Italian Restaurant,American Restaurant,Bakery,Yoga Studio,Seafood Restaurant,Bar,Stationery Store,Fish Market
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Dim Sum Restaurant,Park,Swim School,Bus Line,Wings Joint,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [109]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [113]:
toronto_merged.reset_index(inplace=True)

In [118]:
toronto_merged.drop(['index'],axis=1,inplace=True)

In [119]:
toronto_merged.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Health Food Store,Trail,Pub,Wings Joint,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Bubble Tea Shop,Indian Restaurant,Spa,Juice Bar,Bookstore
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Sandwich Place,Brewery,Fish & Chips Shop,Food & Drink Shop,Sushi Restaurant,Ice Cream Shop,Fast Food Restaurant,Pub,Movie Theater,Italian Restaurant
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Italian Restaurant,American Restaurant,Bakery,Yoga Studio,Seafood Restaurant,Bar,Stationery Store,Fish Market
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Dim Sum Restaurant,Park,Swim School,Bus Line,Wings Joint,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [120]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Bubble Tea Shop,Indian Restaurant,Spa,Juice Bar,Bookstore
2,East Toronto,0,Sandwich Place,Brewery,Fish & Chips Shop,Food & Drink Shop,Sushi Restaurant,Ice Cream Shop,Fast Food Restaurant,Pub,Movie Theater,Italian Restaurant
3,East Toronto,0,Café,Coffee Shop,Italian Restaurant,American Restaurant,Bakery,Yoga Studio,Seafood Restaurant,Bar,Stationery Store,Fish Market
5,Central Toronto,0,Gym,Clothing Store,Food & Drink Shop,Hotel,Breakfast Spot,Sandwich Place,Convenience Store,Park,Gym / Fitness Center,Electronics Store
6,Central Toronto,0,Coffee Shop,Sporting Goods Shop,Clothing Store,Gym / Fitness Center,Mexican Restaurant,Diner,Dessert Shop,Cosmetics Shop,Park,Chinese Restaurant
7,Central Toronto,0,Pizza Place,Sandwich Place,Dessert Shop,Sushi Restaurant,Coffee Shop,Thai Restaurant,Café,Italian Restaurant,Gym,Seafood Restaurant
9,Central Toronto,0,Coffee Shop,Pub,Pizza Place,Fried Chicken Joint,Liquor Store,Sandwich Place,Sports Bar,Restaurant,Bagel Shop,Supermarket
11,Downtown Toronto,0,Coffee Shop,Bakery,Pizza Place,Restaurant,Café,Pub,Italian Restaurant,Chinese Restaurant,Farmers Market,Market
12,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Pub,Bubble Tea Shop,Burger Joint,Café,Men's Store
13,Downtown Toronto,0,Coffee Shop,Park,Pub,Bakery,Theater,Mexican Restaurant,Breakfast Spot,Café,Restaurant,Shoe Store


In [121]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,1,Gym,Playground,Park,Restaurant,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
10,Downtown Toronto,1,Park,Playground,Trail,Wings Joint,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
23,Central Toronto,1,Park,Trail,Jewelry Store,Sushi Restaurant,Wings Joint,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


In [122]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,2,Dim Sum Restaurant,Park,Swim School,Bus Line,Wings Joint,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [123]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,3,Garden,Home Service,Wings Joint,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [124]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,4,Health Food Store,Trail,Pub,Wings Joint,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
